In [255]:
#!pip install tweepy
import pandas as pd
import tweepy

# Scraping

In [187]:
info_api_key = "fSFhW6NJHivhKs88VLtlKQ5t6"
info_api_key_secret = "X7mVc0epNqxlCdRpa4pP2ymRdpbZwnrEr1edW8SG7JhwMgtEnA"
info_bearer_token = "AAAAAAAAAAAAAAAAAAAAAGyulQEAAAAAg7rYt7A8KD2yYFu0datNWtadXOU%3Dou4oAmL2wFiZxrckLd5OCHioEDDRJw4ZEDp7AuuJx7WHIENpGq"

info_client_id = "VzlNTVNlYmw4b21ORUtKdlJiYkc6MTpjaQ"
info_client_secret = "RhKKHZ0bHeq1QqqNsO3Kzw2_dlo0J1strmd9C0n1bziPwn9PiS"

info_acces_token = "1574148627672571904-bIjrsPyaC3zLKaYuF9t7rh4PGkPtwT"
info_acces_token_secret = "jknxWvirBnliMgzINTmmOvGZftZnoShpJe4x1seNBdfth"

info_callback_uri = "https://wibbo.org"
info_website_url = "https://wibbo.org"

In [319]:
def Recuperation_tweets(L_mots,nb):
    client = tweepy.Client(bearer_token=info_bearer_token)

    query = ''
    for m in L_mots:
        query += m + ' OR '
    query=query[:-3]
    query+='lang=en'
    

    #tweets = client.search_recent_tweets(query,max_results=nb,tweet_fields=['context_annotations', 'created_at'], 
                                         #expansions=['author_id'])
    tweets = tweepy.Paginator(client.search_recent_tweets, query=query,
                              tweet_fields=['context_annotations', 'created_at'], expansions=['author_id'],
                              max_results=100).flatten(limit=nb)
    L_tweets=[]
    for tweet in tweets:
        #if len(tweet.context_annotations) > 0: #s'il y a un domaine et l'entité
        L_tweets.append([tweet.data['id'], tweet.data['text'], tweet.data['author_id'], tweet.data['created_at']])
    
    return L_tweets


In [322]:
L=Recuperation_tweets(["suicide","suicidal","I want to die","help","kill myself",],10000)
df=pd.DataFrame(L,columns=['id','text','auteur','date'])


In [327]:
#df.to_csv('Tweepy_Suicide_Tweets.csv',sep=';')
#df.to_csv(r'C:\Users\owen9\OneDrive\Documents\GitHub\TwitterFeelingAnalyse\Dataset\Tweepy_Suicide_Tweets.csv',sep=';',index=False)


# Tests

In [16]:
auth = tweepy.OAuth2BearerHandler(info_bearer_token)
api = tweepy.API(auth)
api

In [90]:
auth = tweepy.OAuth2AppHandler(info_api_key, info_api_key_secret)
api = tweepy.API(auth)
api

In [89]:
auth = tweepy.OAuth2UserHandler(info_api_key, info_api_key_secret, info_acces_token, info_acces_token_secret)
api = tweepy.API(auth)
api

TypeError: __init__() takes 1 positional argument but 5 were given

In [91]:
client = tweepy.Client(info_bearer_token)
client

In [66]:
client.create_tweet(in_reply_to_tweet_id=twitterid[0].id, text='test')

TypeError: Consumer key must be string or bytes, not NoneType

In [22]:
client = tweepy.Client(info_api_key,info_api_key_secret,info_acces_token,info_acces_token_secret)
client

In [93]:
oauth2_user_handler = tweepy.OAuth2UserHandler(
    client_id=info_client_id,
    redirect_uri=info_callback_uri,
    scope=info_scope,
    # Client Secret is only necessary if using a confidential client
    client_secret=info_client_secret
)

print(oauth2_user_handler.get_authorization_url())

https://twitter.com/i/oauth2/authorize?response_type=code&client_id=VzlNTVNlYmw4b21ORUtKdlJiYkc6MTpjaQ&redirect_uri=https%3A%2F%2Ftwitter.com%2F&scope=tweet.read+tweet.write+users.read&state=fE1kMMnR54NV0jZGMNveddWqQvVXzc&code_challenge=3wWCN2wK6T9iFvK4NqWauFq76aG4NJoqqmvJDi8zjjA&code_challenge_method=S256


In [62]:
auth_url='https://twitter.com/home?state=lJ59K5uCQminldrbNR5OX7QmePFTya&code=czRUZEIyeWV4Q2FKR2ZKemdqa1lxei1SdEpLSV8zUmxGTWU1ZkgyaG1iU24tOjE2NzQ2NjM3Nzc1MDA6MToxOmFjOjE'
access_token = oauth2_user_handler.fetch_token(auth_url)

In [63]:
api = tweepy.API(auth)
api

In [ ]:
client = tweepy.Client(bearer_token=info_bearer_token)

# Create a tweet query
# https://github.com/twitterdev/getting-started-with-the-twitter-api-v2-for-academic-research/blob/main/modules/5-how-to-write-search-queries.md
query = 'suicide lang:en'
tweets = client.search_recent_tweets(query, max_results=100)
for tweet in tweets.data:
    print(tweet.text)
    if len(tweet.context_annotations) > 0:
        print(tweet.context_annotations)

In [41]:
client = tweepy.Client(access_token)
client

# Envoyer message

In [69]:
# Authenticate access
auth = tweepy.OAuthHandler(info_api_key, info_api_key_secret)
auth.set_access_token(info_acces_token, info_acces_token_secret)
# Create API handler
api = tweepy.API(auth)

api

In [70]:
twts = api.search(q="Hello World!")

#list of specific strings we want to check for in Tweets
t = ['Hello world!',
    'Hello World!',
    'Hello World!!!',
    'Hello world!!!',
    'Hello, world!',
    'Hello, World!']

for s in twt:
    for i in t:
        if i == s.text:
            sn = s.user.screen_name
            m = "@%s Hello!" % (sn)
            s = api.update_status(m, s.id)

AttributeError: 'API' object has no attribute 'search'

In [72]:
recipient_id = "1618042845625212928"  # ID of the user
api.send_direct_message(recipient_id, "Hoi hoi")

Forbidden: 403 Forbidden
453 - You currently have Essential access which includes access to Twitter API v2 endpoints only. If you need access to this endpoint, you’ll need to apply for Elevated access via the Developer Portal. You can learn more here: https://developer.twitter.com/en/docs/twitter-api/getting-started/about-twitter-api#v2-access-leve

In [96]:
info_api_key = "fSFhW6NJHivhKs88VLtlKQ5t6"
info_api_key_secret = "X7mVc0epNqxlCdRpa4pP2ymRdpbZwnrEr1edW8SG7JhwMgtEnA"
info_bearer_token = "AAAAAAAAAAAAAAAAAAAAAGyulQEAAAAAg7rYt7A8KD2yYFu0datNWtadXOU%3Dou4oAmL2wFiZxrckLd5OCHioEDDRJw4ZEDp7AuuJx7WHIENpGq"

info_client_id = "VzlNTVNlYmw4b21ORUtKdlJiYkc6MTpjaQ"
info_client_secret = "RhKKHZ0bHeq1QqqNsO3Kzw2_dlo0J1strmd9C0n1bziPwn9PiS"

info_acces_token = "1574148627672571904-bIjrsPyaC3zLKaYuF9t7rh4PGkPtwT"
info_acces_token_secret = "jknxWvirBnliMgzINTmmOvGZftZnoShpJe4x1seNBdfth"

info_callback_uri = "https://wibbo.org"
info_website_url = "https://wibbo.org"

info_scope = ["tweet.read","tweet.write","users.read"]

oauth2_user_handler = tweepy.OAuth2UserHandler(
    client_id= info_client_id,
    redirect_uri= info_callback_uri,
    scope= info_scope,
    
    # Client Secret is only necessary if using a confidential client
    client_secret= info_client_secret
)

print(oauth2_user_handler.get_authorization_url())
state = input("Tapez l'url : ")
access_token = oauth2_user_handler.fetch_token(state)
client = tweepy.Client(access_token)

https://twitter.com/i/oauth2/authorize?response_type=code&client_id=VzlNTVNlYmw4b21ORUtKdlJiYkc6MTpjaQ&redirect_uri=https%3A%2F%2Fwibbo.org&scope=tweet.read+tweet.write+users.read&state=fuBMvSWW9qU5lHQTfKPYu4lqTCbxtF&code_challenge=g3gO5zyVAzigXWtRjQ4_z2C_I8IPaNbCW2ZoUsD2xzI&code_challenge_method=S256
Tapez l'url : https://wibbo.org/?state=fuBMvSWW9qU5lHQTfKPYu4lqTCbxtF&code=WDJwNDVNZjBxSTRsSXlGSjZIU2U1MXVHQmZtV3lOVmw0Q2NiM25panMxLW1UOjE2NzQ2ODM5MTUwNTg6MToxOmFjOjE


# Tweets d'un user

In [111]:
tweets = client.get_users_tweets("1059331200261767168")
for tweet in tweets.data:
    print(tweet.text)
    if len(tweet.context_annotations) > 0:
        print(tweet.context_annotations)

RT @PornoxDia: Chanel Preston #StarCards💫
👉https://t.co/1Q7PU9bzsS https://t.co/fcwr0i5KID
RT @PornoxDia: Dillion Harper #StarCards💫
👉https://t.co/qtC3zBLYZA https://t.co/5XWNuM9N5F
RT @PornoxDia: Shawna Lenee #StarCards💫
👉https://t.co/ErKRPCtI53 https://t.co/xEBEzk3TcD
RT @PornoxDia: Julie Cash #StarCards💫
👉https://t.co/tZo7FphUm2 https://t.co/oa7eYE1Vdd
RT @PornoxDia: Ryan Conner #StarCards💫
👉https://t.co/VSI1k7C69C https://t.co/unJXYBuMKP
RT @TeamDanaRico: Soy muy versátil, me gusta bromear, coquetear y especialmente ser travieso y juguetón en mis shows, así que asegúrese de…
RT @TeamDanaRico: ES HORA DE MOSTRAR MI MEJOR PARTE encontraras una chica lista para explorar y fantasear contigo https://t.co/hPzZbuJdh6 h…
RT @TeamDanaRico: El hidrogeno más el oxígeno producen agua, tu y yo producimos placer. https://t.co/eoLTYCv1FZ https://t.co/ZR97vXe6Jx
RT @TeamDanaRico: Ven y disfruta conmigo de interesantes aventuras que te harán perder la cabeza. https://t.co/NdI1zOt9hU https://t.co/sL